In [1]:
import mlflow.keras
import numpy as np
import pandas as pd
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.metrics import Recall, AUC
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve, auc, precision_score, recall_score, f1_score, accuracy_score, confusion_matrix
from tensorflow.keras.callbacks import EarlyStopping
from mlflow.models import infer_signature

In [2]:
metadata = pd.read_csv('dataset/balanced_metadata.csv')
image_folder = 'dataset/balanced_dataset/'
images = []
labels = []
for i, row in metadata.iterrows():
    img_path = os.path.join(image_folder, row['image_name'])
    img = load_img(img_path, target_size=(128, 128))
    images.append(img_to_array(img))
    labels.append(row['target'])

images = np.array(images) / 255.0  # Normalisation
labels = np.array(labels)
print(labels.shape)
print(labels[:10])

(8000,)
[3 3 3 3 3 3 3 3 3 3]


In [3]:
labels = to_categorical(labels, num_classes=4)
print(images.shape)  
print(labels.shape)

(8000, 128, 128, 3)
(8000, 4)


In [4]:
def build_cnn_model(input_shape=(128, 128, 3), num_classes=4, optimizer='adam',
                    dropout_rate=0.5, activation='relu', filters=32, kernel_size=(3, 3)):
    model = Sequential()

    # Première couche convolutionnelle
    model.add(Conv2D(filters, kernel_size, activation=activation, input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))

    # Deuxième couche convolutionnelle
    model.add(Conv2D(filters * 2, kernel_size, activation=activation))
    model.add(MaxPooling2D((2, 2)))

    # Troisième couche convolutionnelle
    model.add(Conv2D(filters * 4, kernel_size, activation=activation))
    model.add(MaxPooling2D((2, 2)))

    # Aplatir les résultats des couches précédentes
    model.add(Flatten())

    # Couches entièrement connectées
    model.add(Dense(128, activation=activation))
    model.add(Dropout(dropout_rate))  # Dropout pour éviter l'overfitting
    model.add(Dense(num_classes, activation='softmax'))  # Classification multi-classe

    # Choisir l'optimiseur basé sur l'argument
    if optimizer == 'adam':
        optimizer_instance = Adam()
    elif optimizer == 'sgd':
        optimizer_instance = SGD()
    elif optimizer == 'rmsprop':
        optimizer_instance = RMSprop()

    # Compiler le modèle avec les métriques supplémentaires
    model.compile(optimizer=optimizer_instance, 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy', Recall(), AUC()])

    return model

In [5]:
# Choix des paramètres
param_grid = {
    'optimizer': 'rmsprop',  # Optimiseur à tester
    'dropout_rate': [0.3, 0.5],  # Taux de dropout
    'activation': 'relu',  # Fonction d'activation des couches
    'filters': [16, 32],  # Nombre de filtres dans les couches convolutionnelles
    'kernel_size': (3, 3),  # Taille des noyaux de convolution
    'batch_size': 32,  # Taille des mini-batchs
    'epochs': 10  # Nombre d'époques
}

''' # Grid search des paramètres
param_grid = {
    'optimizer': ['adam', 'sgd', 'rmsprop'],  # Optimiseur à tester
    'dropout_rate': [0.3, 0.5, 0.7],  # Taux de dropout
    'activation': ['relu', 'tanh', 'elu'],  # Fonction d'activation des couches
    'filters': [32],  # Nombre de filtres dans les couches convolutionnelles
    'kernel_size': [(3, 3)],  # Taille des noyaux de convolution
    'batch_size': [16, 32],  # Taille des mini-batchs
    'epochs': [5, 10]  # Nombre d'époques
}'''

" # Grid search des paramètres\nparam_grid = {\n    'optimizer': ['adam', 'sgd', 'rmsprop'],  # Optimiseur à tester\n    'dropout_rate': [0.3, 0.5, 0.7],  # Taux de dropout\n    'activation': ['relu', 'tanh', 'elu'],  # Fonction d'activation des couches\n    'filters': [32],  # Nombre de filtres dans les couches convolutionnelles\n    'kernel_size': [(3, 3)],  # Taille des noyaux de convolution\n    'batch_size': [16, 32],  # Taille des mini-batchs\n    'epochs': [5, 10]  # Nombre d'époques\n}"

In [6]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

- run on terminal **mlflow server --host 127.0.0.1 --port 5000**

In [2]:
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

- run on terminal **mlflow ui**

In [3]:
mlflow.set_experiment("Brain_Tumor_Classification")

<Experiment: artifact_location='mlflow-artifacts:/344762819335167390', creation_time=1733865490229, experiment_id='344762819335167390', last_update_time=1733865490229, lifecycle_stage='active', name='Brain_Tumor_Classification', tags={}>

In [4]:
experiment = mlflow.get_experiment_by_name("Brain_Tumor_Classification")
experiment_id = experiment.experiment_id

print(f"Experiment ID: {experiment_id}")

Experiment ID: 344762819335167390


In [10]:
# Paramètres pour la validation croisée
num_folds = 3
cv = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

In [11]:
results = []
nb = 4
for filters in param_grid['filters']:
    for dropout_rate in param_grid['dropout_rate']:
        nb = nb + 1

        # Paramètres du modèle
        params = {
            'filters': filters,
            'kernel_size': param_grid['kernel_size'],
            'dropout_rate': dropout_rate,
            'batch_size': param_grid['batch_size'],
            'epochs': param_grid['epochs'],
            'optimizer': param_grid['optimizer']
        }
        print(f"Starting experiment {nb} with parameters: {params}")

        with mlflow.start_run() as run:
            mlflow.set_tag("mlflow.runName", f"CNN : Experiment {nb}")
            mlflow.set_tag("Experiment Info", f"Experiment {nb} for CNN with parameters : {params}")
            mlflow.log_params(params)

            fold_metrics = []
            for fold, (train_idx, val_idx) in enumerate(cv.split(X_train, np.argmax(y_train, axis=1))):
                print(f"Fold {fold + 1}/{num_folds}")

                X_fold_train, X_fold_val = X_train[train_idx], X_train[val_idx]
                y_fold_train, y_fold_val = y_train[train_idx], y_train[val_idx]

                model = build_cnn_model(
                    input_shape=(128, 128, 3),
                    num_classes=4,
                    optimizer=params['optimizer'],
                    dropout_rate=params['dropout_rate'],
                    activation='relu',
                    filters=params['filters'],
                    kernel_size=params['kernel_size']
                )
                early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

                history = model.fit(
                    X_fold_train, y_fold_train,
                    validation_data=(X_fold_val, y_fold_val),
                    batch_size=params['batch_size'],
                    epochs=params['epochs'],
                    callbacks=[early_stopping],
                    verbose=1
                )

                # Prédictions et métriques sur le fold
                y_val_pred = model.predict(X_fold_val)
                y_val_pred_classes = np.argmax(y_val_pred, axis=1)
                y_val_actual_classes = np.argmax(y_fold_val, axis=1)

                accuracy = accuracy_score(y_val_actual_classes, y_val_pred_classes)
                precision = precision_score(y_val_actual_classes, y_val_pred_classes, average='weighted')
                recall = recall_score(y_val_actual_classes, y_val_pred_classes, average='weighted')
                f1 = f1_score(y_val_actual_classes, y_val_pred_classes, average='weighted')
                auc_roc = roc_auc_score(y_fold_val, y_val_pred, multi_class='ovr')

                fold_metrics.append({
                    'accuracy': accuracy,
                    'precision': precision,
                    'recall': recall,
                    'f1': f1,
                    'auc_roc': auc_roc
                })

                # Logging des métriques pour chaque fold
                mlflow.log_metric(f"fold_{fold + 1}_accuracy", accuracy)
                mlflow.log_metric(f"fold_{fold + 1}_precision", precision)
                mlflow.log_metric(f"fold_{fold + 1}_recall", recall)
                mlflow.log_metric(f"fold_{fold + 1}_f1", f1)
                mlflow.log_metric(f"fold_{fold + 1}_auc_roc", auc_roc)

            # Moyennes des métriques sur tous les folds
            avg_accuracy = np.mean([m['accuracy'] for m in fold_metrics])
            avg_precision = np.mean([m['precision'] for m in fold_metrics])
            avg_recall = np.mean([m['recall'] for m in fold_metrics])
            avg_f1 = np.mean([m['f1'] for m in fold_metrics])
            avg_auc_roc = np.mean([m['auc_roc'] for m in fold_metrics])

            mlflow.log_metric("avg_accuracy", avg_accuracy)
            mlflow.log_metric("avg_precision", avg_precision)
            mlflow.log_metric("avg_recall", avg_recall)
            mlflow.log_metric("avg_f1", avg_f1)
            mlflow.log_metric("avg_auc_roc", avg_auc_roc)

            # Prédictions finales sur X_test
            y_test_pred = model.predict(X_test)
            y_test_pred_classes = np.argmax(y_test_pred, axis=1)
            y_test_actual_classes = np.argmax(y_test, axis=1)

            # Calcul des métriques sur le test
            test_accuracy = accuracy_score(y_test_actual_classes, y_test_pred_classes)
            test_precision = precision_score(y_test_actual_classes, y_test_pred_classes, average='weighted')
            test_recall = recall_score(y_test_actual_classes, y_test_pred_classes, average='weighted')
            test_f1 = f1_score(y_test_actual_classes, y_test_pred_classes, average='weighted')

            mlflow.log_metric("test_accuracy", test_accuracy)
            mlflow.log_metric("test_precision", test_precision)
            mlflow.log_metric("test_recall", test_recall)
            mlflow.log_metric("test_f1", test_f1)

            predictions_df = pd.DataFrame({"Actual": y_test_actual_classes, "Predicted": y_test_pred_classes})
            predictions_csv_path = "tmp/predictions.csv"
            predictions_df.to_csv(predictions_csv_path, index=False)
            mlflow.log_artifact(predictions_csv_path, artifact_path="predictions")

            # Matrice de confusion
            matrix = confusion_matrix(y_test_actual_classes, y_test_pred_classes)
            plt.figure(figsize=(8, 6))
            sns.heatmap(matrix, annot=True, fmt='d', cmap='Blues')
            plt.title("Confusion Matrix")
            cf_matrix_path = "tmp/confusion_matrix.png"
            plt.savefig(cf_matrix_path)
            plt.close()
            mlflow.log_artifact(cf_matrix_path, artifact_path="cf_matrix")

            # Courbe ROC AUC
            fpr, tpr, _ = roc_curve(y_test.ravel(), y_test_pred.ravel())
            roc_auc = auc(fpr, tpr)

            plt.figure()
            plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
            plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
            plt.xlabel("False Positive Rate")
            plt.ylabel("True Positive Rate")
            plt.title("Receiver Operating Characteristic")
            plt.legend(loc="lower right")
            roc_curve_path = "tmp/roc_curve.png"
            plt.savefig(roc_curve_path)
            plt.close()
            mlflow.log_artifact(roc_curve_path, artifact_path="roc_curve")

            # Enregistrement du modèle final
            mlflow.keras.log_model(model, "cnn_model")
            signature = infer_signature(X_train, model.predict(X_train))
            model_info = mlflow.sklearn.log_model(
                sk_model=model,
                artifact_path="cnn_model",
                signature=signature,
                input_example=X_train[:1],
                registered_model_name="convolutional-neural-network",
            )


Starting experiment 5 with parameters: {'filters': 16, 'kernel_size': (3, 3), 'dropout_rate': 0.3, 'batch_size': 32, 'epochs': 10, 'optimizer': 'rmsprop'}
Fold 1/3


c:\Users\user\Documents\GitHub\ML-Excuses\env\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 10s 84ms/step - accuracy: 0.4786 - auc: 0.7424 - loss: 1.2509 - recall: 0.1993 - val_accuracy: 0.6550 - val_auc: 0.8808 - val_loss: 0.8506 - val_recall: 0.5958
Epoch 2/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 8s 76ms/step - accuracy: 0.7186 - auc: 0.9147 - loss: 0.7003 - recall: 0.6458 - val_accuracy: 0.7903 - val_auc: 0.9532 - val_loss: 0.5176 - val_recall: 0.7663
Epoch 3/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 8s 76ms/step - accuracy: 0.8038 - auc: 0.9513 - loss: 0.5264 - recall: 0.7657 - val_accuracy: 0.8342 - val_auc: 0.9664 - val_loss: 0.4289 - val_recall: 0.8125
Epoch 4/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 8s 79ms/step - accuracy: 0.8368 - auc: 0.9688 - loss: 0.4128 - recall: 0.8157 - val_accuracy: 0.8207 - val_auc: 0.9633 - val_loss: 0.4630 - val_recall: 0.8079
Epoch 5/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 8s 75ms/step - accuracy: 0.8689 - auc: 0.9775 - loss: 0.3486 - recall: 0.8592 - val_accuracy: 0.8576 - val_auc: 0.9732 - val_loss: 0.3788 - val_recall: 0.8483
Epoc

c:\Users\user\Documents\GitHub\ML-Excuses\env\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


107/107 ━━━━━━━━━━━━━━━━━━━━ 9s 79ms/step - accuracy: 0.4554 - auc_1: 0.7430 - loss: 1.1796 - recall_1: 0.2304 - val_accuracy: 0.6719 - val_auc_1: 0.9015 - val_loss: 0.7575 - val_recall_1: 0.6397
Epoch 2/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 8s 77ms/step - accuracy: 0.7383 - auc_1: 0.9228 - loss: 0.6656 - recall_1: 0.6795 - val_accuracy: 0.7827 - val_auc_1: 0.9470 - val_loss: 0.5510 - val_recall_1: 0.7639
Epoch 3/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 8s 76ms/step - accuracy: 0.7855 - auc_1: 0.9498 - loss: 0.5374 - recall_1: 0.7599 - val_accuracy: 0.8278 - val_auc_1: 0.9647 - val_loss: 0.4534 - val_recall_1: 0.8079
Epoch 4/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 8s 75ms/step - accuracy: 0.8487 - auc_1: 0.9717 - loss: 0.3964 - recall_1: 0.8260 - val_accuracy: 0.8530 - val_auc_1: 0.9697 - val_loss: 0.4142 - val_recall_1: 0.8389
Epoch 5/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 8s 77ms/step - accuracy: 0.8923 - auc_1: 0.9837 - loss: 0.2917 - recall_1: 0.8832 - val_accuracy: 0.8723 - val_auc_1: 0.9777 - val_loss: 0.3546

...

160/160 ━━━━━━━━━━━━━━━━━━━━ 8s 50ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step


Registered model 'convolutional-neural-network' already exists. Creating a new version of this model...
2024/12/11 03:31:03 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: convolutional-neural-network, version 8


🏃 View run CNN : Experiment 8 at: http://127.0.0.1:5000/#/experiments/344762819335167390/runs/9af40823da7b4311a2d37e347feeb674
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/344762819335167390


Created version '8' of model 'convolutional-neural-network'.


In [7]:
runs = mlflow.search_runs(experiment_ids=experiment_id)

best_cnn = runs.sort_values(by="metrics.test_accuracy", ascending=False).iloc[0]

best_cnn_id = best_cnn["run_id"]

print(f"Best Run ID: {best_cnn_id}")
print(f"Best Run Metrics: {best_cnn[['metrics.avg_accuracy', 'metrics.avg_f1', 'metrics.test_accuracy', 'metrics.test_f1']]}")

best_cnn_uri = f"runs:/{best_cnn_id}/cnn_model"

best_cnn_uri

Best Run ID: 7b6ab6dae00443878f8e89536fcb2bd1
Best Run Metrics: metrics.avg_accuracy     0.911528
metrics.avg_f1           0.911026
metrics.test_accuracy    0.914375
metrics.test_f1          0.914609
Name: 5, dtype: object


'runs:/7b6ab6dae00443878f8e89536fcb2bd1/cnn_model'

In [8]:
runs

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.fold_2_recall,metrics.fold_2_precision,metrics.fold_1_accuracy,metrics.fold_3_auc_roc,...,params.batch_size,params.optimizer,params.kernel_size,params.epochs,tags.mlflow.runName,tags.Experiment Info,tags.mlflow.user,tags.mlflow.source.type,tags.mlflow.log-model.history,tags.mlflow.source.name
0,9af40823da7b4311a2d37e347feeb674,344762819335167390,FINISHED,mlflow-artifacts:/344762819335167390/9af40823d...,2024-12-11 02:19:27.012000+00:00,2024-12-11 02:31:03.224000+00:00,0.873462,0.874331,0.882835,0.985932,...,32,rmsprop,"(3, 3)",10,CNN : Experiment 8,Experiment 8 for CNN with parameters : {'filte...,user,LOCAL,"[{""run_id"": ""9af40823da7b4311a2d37e347feeb674""...",c:\Users\user\Documents\GitHub\ML-Excuses\env\...
1,4f91d4bdd3e248269818fc844b689035,344762819335167390,FINISHED,mlflow-artifacts:/344762819335167390/4f91d4bdd...,2024-12-11 02:06:00.797000+00:00,2024-12-11 02:19:26.992000+00:00,0.888108,0.889865,0.898653,0.981860,...,32,rmsprop,"(3, 3)",10,CNN : Experiment 7,Experiment 7 for CNN with parameters : {'filte...,user,LOCAL,"[{""run_id"": ""4f91d4bdd3e248269818fc844b689035""...",c:\Users\user\Documents\GitHub\ML-Excuses\env\...
2,8e0191b3db8a480baaa5e8efb599c86b,344762819335167390,FINISHED,mlflow-artifacts:/344762819335167390/8e0191b3d...,2024-12-11 02:01:13.851000+00:00,2024-12-11 02:06:00.779000+00:00,0.903339,0.904348,0.888108,0.982342,...,32,rmsprop,"(3, 3)",10,CNN : Experiment 6,Experiment 6 for CNN with parameters : {'filte...,user,LOCAL,"[{""run_id"": ""8e0191b3db8a480baaa5e8efb599c86b""...",c:\Users\user\Documents\GitHub\ML-Excuses\env\...
3,e9c7c05b3a9a44eab98575869625bfcb,344762819335167390,FINISHED,mlflow-artifacts:/344762819335167390/e9c7c05b3...,2024-12-11 01:56:48.062000+00:00,2024-12-11 02:01:13.829000+00:00,0.883421,0.883450,0.882835,0.980024,...,32,rmsprop,"(3, 3)",10,CNN : Experiment 5,Experiment 5 for CNN with parameters : {'filte...,user,LOCAL,"[{""run_id"": ""e9c7c05b3a9a44eab98575869625bfcb""...",c:\Users\user\Documents\GitHub\ML-Excuses\env\...
4,c1e700c88edc48c69caf79d0412ba5f1,344762819335167390,RUNNING,mlflow-artifacts:/344762819335167390/c1e700c88...,2024-12-11 01:56:22.147000+00:00,NaT,NaN,NaN,NaN,NaN,...,32,rmsprop,"(3, 3)",10,CNN : Experiment 0,Experiment 5 for CNN with parameters : {'filte...,user,LOCAL,None,c:\Users\user\Documents\GitHub\ML-Excuses\env\...
5,7b6ab6dae00443878f8e89536fcb2bd1,344762819335167390,FINISHED,mlflow-artifacts:/344762819335167390/7b6ab6dae...,2024-12-10 22:35:58.683000+00:00,2024-12-10 22:47:32.615000+00:00,0.888108,0.890458,0.913884,0.987324,...,32,adam,"(3, 3)",10,CNN : Experiment 4,Experiment 4 for CNN with parameters : {'filte...,user,LOCAL,"[{""run_id"": ""7b6ab6dae00443878f8e89536fcb2bd1""...",c:\Users\user\Documents\GitHub\ML-Excuses\env\...
6,e8c4caba61494315b0c7cfeaccf76c72,344762819335167390,FINISHED,mlflow-artifacts:/344762819335167390/e8c4caba6...,2024-12-10 22:25:28.632000+00:00,2024-12-10 22:35:58.665000+00:00,0.869947,0.869288,0.899238,0.983052,...,32,adam,"(3, 3)",10,CNN : Experiment 3,Experiment 3 for CNN with parameters : {'filte...,user,LOCAL,"[{""run_id"": ""e8c4caba61494315b0c7cfeaccf76c72""...",c:\Users\user\Documents\GitHub\ML-Excuses\env\...
7,0691c41a1bc54dbcb837edc3462cbbd1,344762819335167390,FINISHED,mlflow-artifacts:/344762819335167390/0691c41a1...,2024-12-10 22:20:57.271000+00:00,2024-12-10 22:25:28.612000+00:00,0.890451,0.890898,0.900996,0.981090,...,32,adam,"(3, 3)",10,CNN : Experiment 2,Experiment 2 for CNN with parameters : {'filte...,user,LOCAL,"[{""run_id"": ""0691c41a1bc54dbcb837edc3462cbbd1""...",c:\Users\user\Documents\GitHub\ML-Excuses\env\...
8,ebf5c6e3ba6e459ba70ae803999bd4da,344762819335167390,FINISHED,mlflow-artifacts:/344762819335167390/ebf5c6e3b...,2024-12-10 22:14:35.155000+00:00,2024-12-10 22:20:57.252000+00:00,0.896309,0.896123,0.899824,0.982114,...,32,adam,"(3, 3)",10,CNN : Experiment 1,Experiment 1 for CNN with parameters : {'filte...,user,LOCAL,"[{

In [13]:
mlflow.register_model(
        model_uri=best_cnn_uri,
        name="BrainTumor_BestCNN"
    )

Successfully registered model 'BrainTumor_BestCNN'.
2024/12/11 03:31:03 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: BrainTumor_BestCNN, version 1
Created version '1' of model 'BrainTumor_BestCNN'.


<ModelVersion: aliases=[], creation_timestamp=1733884263385, current_stage='None', description='', last_updated_timestamp=1733884263385, name='BrainTumor_BestCNN', run_id='7b6ab6dae00443878f8e89536fcb2bd1', run_link='', source='mlflow-artifacts:/344762819335167390/7b6ab6dae00443878f8e89536fcb2bd1/artifacts/cnn_model', status='READY', status_message='', tags={}, user_id='', version='1'>